COMBINE sammanställningsresultat för 2020-2024

In [ ]:
years = range(2022, 2024)
files = [f"data/yh-sokande-{year}-behorighet-reell-kompetens.xlsx" for year in years]

df_list = []

for file in files:
    year = file.split('-')[-1].split('.')[0]  # Extract year from filename
    try:
        df = pd.read_excel(file, sheet_name='Sökande 2024 - Tab1', header=5)
        df['År'] = int(year)  # Add year column
        df_list.append(df)
    except Exception as e:
        print(f"Error processing {file}: {e}")

# Combine all data
combined_df = pd.concat(df_list, ignore_index=True)

# Save to new Excel file
combined_df.to_excel("sammanstallning-resultat-2023-2024.xlsx", index=False)

In [ ]:
import pandas as pd

df1 = pd.read_excel("data/sammanstallning-resultat-2022-2024.xlsx")
df2 = pd.read_excel("data/sammanstallning-resultat-2023-2024.xlsx")

# Combine the data
combined_df = pd.concat([df1, df2], ignore_index=True)

combined_df.to_excel("sammanstallning-resultat-2020-2024-updated.xlsx", index=False)

## Age & gender EDA - Sökande till yrkeshögskolan

In [ ]:
import pandas as pd

excel_path = 'data/Utbildningsansökning_age.xlsx'

age_df = pd.read_excel(excel_path, sheet_name='Age')
edu_df = pd.read_excel(excel_path, sheet_name='Education')

print(age_df.info())
print(edu_df.info())


### EDA - Antalet studerande i YH inom olika utbildningsområden 2012-2024

In [4]:
df3 = pd.read_excel("data/Antalet studerande i YH inom olika utbildningsområden 2012-2024.xlsx")
df3.head()

,År,Data/It,"Ekonomi, administration och försäljning",Friskvård och kroppsvård,"Hotell, restaurang och turism",Hälso- och sjukvård samt socialt arbete,Journalistik och information,Juridik,"Kultur, media och design","Lantbruk, djurvård, trädgård, skog och fiske",Pedagogik och undervisning,Samhällsbyggnad och byggteknik,Säkerhetstjänster,Teknik och tillverkning,Transporttjänster,Övrigt,Totalt
0,2005,2587,6238,481,2223,2992,114,..,2803,920,16,1406,267,3920,822,..,24789
1,2006,2546,7582,738,2553,3807,101,61,2938,1341,103,1947,425,4310,1104,..,29556
2,2007,2797,8807,883,2906,4200,114,122,2893,1538,186,2388,565,4928,1357,..,33684
3,2008,2817,10264,993,3074,4029,159,157,2993,1502,220,2641,626,5563,1590,29,36657
4,2009,2857,11286,1009,2965,4091,213,167,2952,1549,250,3183,629,6511,1696,59,39417


In [12]:
#df3.replace('..', pd.NA, inplace=True)
#df3.to_excel("cleaned_dataset.xlsx", index=False)

# Get number of students in Data/IT in 2005
data_it_2005 = df3.loc[df3['År'] == 2005, 'Data/It'].values[0]
data_it_2005



np.int64(2587)

In [25]:
import duckdb 

test = duckdb.query("""
SELECT "Data/It"
FROM df3
WHERE År = 2005
""").df()

test




,Data/It
0,2587


In [ ]:
import pandas as pd
import plotly.express as px

df = pd.read_excel("data/Utbildningsansökning_age.xlsx", sheet_name='Education')

# Clean and convert numbers (remove spaces and convert to int)
for col in ['Total', 'Women', 'Men']:
    df[col] = df[col].astype(str).str.replace(' ', '').astype(int)

# Filter for the most recent year (2024)
df_2024 = df[df['Year'] == 2024]

# Create the overlapping horizontal bar chart
fig = px.bar(
    df_2024.melt(id_vars=['Education'], value_vars=['Women', 'Men'], var_name='Gender', value_name='Applicants'),
    x='Applicants',
    y='Education',
    color='Gender',
    barmode='overlay',
    orientation='h',
    title='Men vs Women Applicants per Education Field (2024)'
)

fig.update_layout(
    height=700,
    xaxis_title='Number of Applicants',
    yaxis_title='Education Field',
    legend_title='Gender'
)

fig.show()


In [49]:
import pandas as pd
import plotly.express as px

df = pd.read_excel("data/Utbildningsansökning_age.xlsx", sheet_name='Education')

# Clean and convert numbers
for col in ['Total', 'Women', 'Men']:
    df[col] = df[col].astype(str).str.replace(' ', '').astype(int)

# Filter for the years 2023 and 2024
df_filtered = df[df['Year'].isin([2023, 2024])].copy()  # Create a copy to avoid the warning

# Calculate total applicants per Education for both years
df_filtered['TotalApplicants'] = df_filtered['Women'] + df_filtered['Men']

# Group by Education and sum the applicants from both years
df_grouped = df_filtered.groupby('Education').agg(
    Women=('Women', 'sum'),
    Men=('Men', 'sum'),
    TotalApplicants=('TotalApplicants', 'sum')
).reset_index()

# Get top 10 education fields by total applicants
top_10 = df_grouped.sort_values(by='TotalApplicants', ascending=True).tail(10)

# Melt data for plotting
melted = top_10.melt(
    id_vars=['Education'],
    value_vars=['Women', 'Men'],
    var_name='Gender',
    value_name='Applicants'
)

# Create the bar chart
fig = px.bar(
    melted,
    x='Applicants',
    y='Education',
    color='Gender',
    barmode='overlay',
    orientation='h',
    title='Men vs Women Applicants per Education Field (2023 & 2024) - Top 10',
    color_discrete_map={
        'Women': 'lightblue', 
        'Men': 'grey'  
    },
)

fig.update_layout(
    height=700,
    xaxis_title='',
    yaxis_title='',
    legend_title='Gender', 
    paper_bgcolor='white',   # Outside the plot area
    plot_bgcolor='white',     # Inside the plot area (behind bars)
    title=dict(
        text='Men vs Women Applicants per Education Field (2023 & 2024) - Top 10',
        font=dict(color='#334850', size=20, )
    ),

    xaxis=dict(
        showline=True,
        linewidth=1,
        linecolor='black',
        mirror=False,
        ticks='outside',
        showticklabels=True
    ),
    yaxis=dict(
        tickfont=dict(color='#334850', size=13,)),
        #Hoverlabel settings
        hoverlabel=dict(
        font=dict(
            color='white',      # Text color
            family='Inter',     # Optional: custom font
            size=15             # Font size
        ),
        bgcolor='darkblue',     # Background color of the hover box
        bordercolor='black'     # Border color (optional)
    )
)

fig.show()


## Pie chart for Age ranges

In [ ]:
import pandas as pd
import plotly.express as px

# Read the data
df = pd.read_excel("data/Utbildningsansökning_age.xlsx", sheet_name="Age")
df.columns = df.columns.str.strip()

# Reshape data for grouped bar chart
df_melted = df.melt(id_vars=["Age groups"], value_vars=["Women", "Men"],
                    var_name="Gender", value_name="Applications")

# Create grouped bar chart
fig = px.bar(
    df_melted,
    x="Age groups",
    y="Applications",
    color="Gender",
    barmode="group",
    title="Utbildningsansökningar 2024: Kvinnor vs. Män per åldersgrupp",
    labels={"Applications": "Antal ansökningar"}
)

fig.show()
